In [25]:
import numpy as np
import timeit
import argparse
import cv2
import matplotlib.pyplot as plt
import math
import queue
from Obstacle import *

In [26]:
threshold = 0.5
visited = np.zeros((int(400/threshold),int(300/threshold),12), dtype='int')

In [27]:
def halfRound(n):
    return round(2*n)/2

In [28]:
def checkAndStoreInVisited(x, y, theta, threshold=0.5, thetaStep=30):
    result = False
    x = int(halfRound(x)/threshold)
    y = int(halfRound(y)/threshold)
    theta = int(theta/thetaStep)
    if(visited[x,y,theta] == 0):
        visited[x,y,theta] = 1
        result = True
    return result